In [1]:
from models import GCN
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 100
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [3]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = GCN(in_feats, 16, n_classes, 2, activation, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

In [6]:
train(model, g, features, labels, train_mask, val_mask, test_mask)

Epoch 01, Loss: 1.0990, Train: 0.3500, Val: 0.4180, Test: 0.4070
Epoch 02, Loss: 1.0905, Train: 0.8833, Val: 0.6620, Test: 0.6140
Epoch 03, Loss: 1.0785, Train: 0.9333, Val: 0.7380, Test: 0.7270
Epoch 04, Loss: 1.0614, Train: 0.9500, Val: 0.7440, Test: 0.7160
Epoch 05, Loss: 1.0457, Train: 0.9167, Val: 0.7180, Test: 0.6940
Epoch 06, Loss: 1.0247, Train: 0.8833, Val: 0.7180, Test: 0.6690
Epoch 07, Loss: 1.0093, Train: 0.8833, Val: 0.7220, Test: 0.6730
Epoch 08, Loss: 0.9861, Train: 0.9167, Val: 0.7260, Test: 0.6760
Epoch 09, Loss: 0.9546, Train: 0.9167, Val: 0.7280, Test: 0.6920
Epoch 10, Loss: 0.9339, Train: 0.9500, Val: 0.7200, Test: 0.7110
Epoch 11, Loss: 0.9159, Train: 0.9500, Val: 0.7360, Test: 0.7150
Epoch 12, Loss: 0.8898, Train: 0.9500, Val: 0.7460, Test: 0.7130
Epoch 13, Loss: 0.8682, Train: 0.9500, Val: 0.7460, Test: 0.7030
Epoch 14, Loss: 0.8395, Train: 0.9333, Val: 0.7360, Test: 0.7080
Epoch 15, Loss: 0.8020, Train: 0.9333, Val: 0.7300, Test: 0.7070
Epoch 16, Loss: 0.7738, T